In [3]:
import ee
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Initialize
ee.Initialize(project="kolkata-flood-mapping")

# Fix: Check where you actually are
print(f"Current directory: {Path.cwd()}")

# Adjust based on where you are
# If you're in notebooks/:
if Path.cwd().name == 'notebooks':
    PROJECT_ROOT = Path.cwd().parent
    DATA_DIR = PROJECT_ROOT / 'data'
# If you're already in project root:
elif (Path.cwd() / 'data').exists():
    PROJECT_ROOT = Path.cwd()
    DATA_DIR = PROJECT_ROOT / 'data'
# If you're somewhere else:
else:
    # Hardcode for safety
    PROJECT_ROOT = Path('/Users/romitbasak/Projects/KolkataFloodMapping')
    DATA_DIR = PROJECT_ROOT / 'data'

SAR_DIR = DATA_DIR / 'sar'
FEATURES_DIR = DATA_DIR / 'features'
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 60)
print("SAR FEATURE EXTRACTION PIPELINE - SETUP")
print("=" * 60)

print(f"Paths verified:")
print(f"  Project root: {PROJECT_ROOT}")
print(f"  Data dir: {DATA_DIR}")
print(f"  Wards file: {DATA_DIR / 'wards/kmc_wards_gee_ready.geojson'}")
print(f"  Exists: {(DATA_DIR / 'wards/kmc_wards_gee_ready.geojson').exists()}")

# Load wards
wards = gpd.read_file(DATA_DIR / 'wards/kmc_wards_gee_ready.geojson')
wards['WARD'] = wards['WARD'].astype(str)

kmc_bounds = wards.total_bounds
kmc_bbox = ee.Geometry.Rectangle([kmc_bounds[0], kmc_bounds[1], kmc_bounds[2], kmc_bounds[3]])

print(f"\n✓ {len(wards)} wards loaded")

# Load dry dates
rainfall_df = pd.read_csv(SAR_DIR / 'sar_5day_rainfall_nov_apr.csv')
dry_dates_millis = rainfall_df[rainfall_df['is_dry']]['date_millis'].tolist()

print(f"✓ {len(dry_dates_millis)} verified dry dates")

# Otsu threshold
OTSU_S1 = -14.90
print(f"✓ Otsu: {OTSU_S1:.2f} dB")

# Temporal periods
periods = {
    '2014-2016': ('2014-11-01', '2017-04-30'),
    '2017-2019': ('2017-11-01', '2020-04-30'),
    '2020-2022': ('2020-11-01', '2023-04-30'),
    '2023-2025': ('2023-11-01', '2025-11-30')
}

def get_period_for_date(date_str):
    for period_name, (start, end) in periods.items():
        if start <= date_str <= end:
            return period_name
    return None

print(f"✓ Period assignment ready")
print(f"\n🎯 Ready for SAR extraction!")

Current directory: /Users/romitbasak/Projects/KolkataFloodMapping/data/sar
SAR FEATURE EXTRACTION PIPELINE - SETUP
Paths verified:
  Project root: /Users/romitbasak/Projects/KolkataFloodMapping
  Data dir: /Users/romitbasak/Projects/KolkataFloodMapping/data
  Wards file: /Users/romitbasak/Projects/KolkataFloodMapping/data/wards/kmc_wards_gee_ready.geojson
  Exists: True

✓ 141 wards loaded
✓ 565 verified dry dates
✓ Otsu: -14.90 dB
✓ Period assignment ready

🎯 Ready for SAR extraction!


In [5]:
print("=" * 60)
print("TEST: SAR FEATURE EXTRACTION (10 DATES)")
print("=" * 60)

# Load Sentinel-1
s1_full = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(kmc_bbox) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .filterDate('2014-08-01', '2025-11-30') \
    .select('VV')

all_s1_dates = s1_full.aggregate_array('system:time_start').distinct().getInfo()

print(f"✓ Total S1 dates: {len(all_s1_dates)}")

# Sample 10 dates
test_indices = np.linspace(0, len(all_s1_dates)-1, 10, dtype=int)
test_dates = [all_s1_dates[i] for i in test_indices]

print(f"✓ Testing 10 sample dates\n")

# Test extraction (KMC-wide stats only, no ward geometries)
test_results = []

for i, date_millis in enumerate(test_dates, 1):
    date_obj = datetime.fromtimestamp(date_millis / 1000)
    date_str = date_obj.strftime('%Y-%m-%d')
    period = get_period_for_date(date_str)

    print(f"  [{i}/10] {date_str} ({period})...", end='', flush=True)

    try:
        # Get image
        s1_img = s1_full.filter(ee.Filter.eq('system:time_start', int(date_millis))).first()

        # Apply Otsu
        vv_filt = s1_img.select('VV').focalMedian(100, 'circle', 'meters')
        water = vv_filt.lt(OTSU_S1)

        # Get KMC-wide statistics (no ward geometries)
        stats = water.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=kmc_bbox,
            scale=10,
            maxPixels=1e9
        ).getInfo()

        water_fraction = stats.get('VV', 0)

        test_results.append({
            'date': date_str,
            'date_millis': date_millis,
            'period': period,
            'kmc_water_fraction': water_fraction
        })

        print(f" ✓ Water={water_fraction:.3f}")

    except Exception as e:
        print(f" ❌ {e}")

test_df = pd.DataFrame(test_results)

print(f"\n✅ Test complete!")
print(f"   Successful: {len(test_df)} / 10")

if len(test_df) > 0:
    print(f"\n📊 Results:")
    print(test_df[['date', 'period', 'kmc_water_fraction']])

    print(f"\n🎯 Pipeline validated!")
    print(f"   Next: Process all dates with ward-level export approach")
    print(f"   (Export SAR extent rasters, process locally like permanent masks)")
else:
    print(f"\n⚠️  All failed - need to debug")

TEST: SAR FEATURE EXTRACTION (10 DATES)
✓ Total S1 dates: 792
✓ Testing 10 sample dates

  [1/10] 2014-10-15 (None)... ✓ Water=0.236
  [2/10] 2017-05-13 (None)... ✓ Water=0.074
  [3/10] 2018-07-19 (2017-2019)... ✓ Water=0.072
  [4/10] 2019-09-19 (2017-2019)... ✓ Water=0.209
  [5/10] 2020-09-13 (None)... ✓ Water=0.216
  [6/10] 2021-09-01 (2020-2022)... ✓ Water=0.072
  [7/10] 2022-08-27 (2020-2022)... ✓ Water=0.001
  [8/10] 2023-10-09 (None)... ✓ Water=0.066
  [9/10] 2024-12-02 (2023-2025)... ✓ Water=0.001
  [10/10] 2020-07-21 (None)... ✓ Water=0.192

✅ Test complete!
   Successful: 10 / 10

📊 Results:
         date     period  kmc_water_fraction
0  2014-10-15       None            0.235583
1  2017-05-13       None            0.074075
2  2018-07-19  2017-2019            0.072352
3  2019-09-19  2017-2019            0.209226
4  2020-09-13       None            0.216213
5  2021-09-01  2020-2022            0.072243
6  2022-08-27  2020-2022            0.000792
7  2023-10-09       None        

In [6]:
print("=" * 60)
print("BATCH TEST: 100 DATES × 141 WARDS")
print("=" * 60)

print("🎯 Strategy: Export to Drive, process locally")
print("   Avoids geometry issues + Mac-friendly!")
print("   Expected time: 1-2 hours\n")

# Sample 100 dates (spread across all years)
batch_size = 100
batch_indices = np.linspace(0, len(all_s1_dates)-1, batch_size, dtype=int)
batch_dates = [all_s1_dates[i] for i in batch_indices]

print(f"Selected {len(batch_dates)} dates")
print(f"  Range: {datetime.fromtimestamp(batch_dates[0]/1000).strftime('%Y-%m-%d')} to {datetime.fromtimestamp(batch_dates[-1]/1000).strftime('%Y-%m-%d')}")

# Export water extent for each date
print(f"\n⚙️  Exporting SAR water extent rasters to Drive...")
print(f"   This will take 45-60 min (background processing)")

export_count = 0

for date_millis in batch_dates[:10]:  # Start with just 10 for tonight
    date_obj = datetime.fromtimestamp(date_millis / 1000)
    date_str = date_obj.strftime('%Y%m%d')

    # Get image
    s1_img = s1_full.filter(ee.Filter.eq('system:time_start', int(date_millis))).first()

    # Apply Otsu
    vv_filt = s1_img.select('VV').focalMedian(100, 'circle', 'meters')
    water = vv_filt.lt(OTSU_S1).toByte()  # 0 or 1

    # Export
    task = ee.batch.Export.image.toDrive(
        image=water,
        description=f'sar_water_{date_str}',
        folder='Earth_Engine_Exports',
        fileNamePrefix=f'sar_water_extent_{date_str}',
        region=kmc_bbox.getInfo()['coordinates'],
        scale=10,
        maxPixels=1e10,
        fileFormat='GeoTIFF'
    )

    task.start()
    export_count += 1

    if (export_count) % 5 == 0:
        print(f"  Started {export_count} exports...")

print(f"\n✅ {export_count} exports started!")
print(f"\n💡 For tonight: Starting with 10 dates (validation)")
print(f"   Tomorrow: Scale to 100, then full 1,989")
print(f"\n⏰ Check GEE Tasks in 45-60 min, then run next cell!")

BATCH TEST: 100 DATES × 141 WARDS
🎯 Strategy: Export to Drive, process locally
   Avoids geometry issues + Mac-friendly!
   Expected time: 1-2 hours

Selected 100 dates
  Range: 2014-10-15 to 2020-07-21

⚙️  Exporting SAR water extent rasters to Drive...
   This will take 45-60 min (background processing)
  Started 5 exports...
  Started 10 exports...

✅ 10 exports started!

💡 For tonight: Starting with 10 dates (validation)
   Tomorrow: Scale to 100, then full 1,989

⏰ Check GEE Tasks in 45-60 min, then run next cell!


In [9]:
print("=" * 60)
print("FIXED PROCESSING: CRS REPROJECTION")
print("=" * 60)

import rasterio
from rasterio.mask import mask as rasterio_mask

# Get SAR files
sar_files = sorted(SAR_DIR.glob('sar_water_extent_*.tif'))

print(f"Processing {len(sar_files)} files...")

all_features = []

for sar_file in sar_files:
    # Extract date
    date_str = sar_file.stem.split('_')[-1]  # YYYYMMDD
    date_formatted = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
    period = get_period_for_date(date_formatted)

    with rasterio.open(sar_file) as src:
        # Reproject wards to match raster CRS
        wards_reprojected = wards.to_crs(src.crs)

        # Process each ward
        for idx, ward_row in wards_reprojected.iterrows():
            ward_id = ward_row['WARD']
            ward_geom = [ward_row['geometry'].__geo_interface__]

            try:
                # Mask to ward
                out_img, _ = rasterio_mask(src, ward_geom, crop=True, filled=False)
                data = out_img[0]

                # Remove masked values
                if hasattr(data, 'mask'):
                    data = data[~data.mask]

                data = data[np.isfinite(data)]

                if len(data) > 0:
                    water_pixels = (data == 1).sum()
                    total_pixels = len(data)
                    water_fraction = water_pixels / total_pixels

                    all_features.append({
                        'date': date_formatted,
                        'ward_id': ward_id,
                        'period': period,
                        'sar_water_extent': water_fraction,
                        'water_pixels': int(water_pixels),
                        'total_pixels': int(total_pixels)
                    })
            except:
                pass

    print(f"  ✓ {date_formatted} ({period})")

# Create DataFrame
sar_features = pd.DataFrame(all_features)

print(f"\n✅ Processed!")
print(f"   Total rows: {len(sar_features)}")
print(f"   Dates: {sar_features['date'].nunique()}")
print(f"   Wards: {sar_features['ward_id'].nunique()}")

print(f"\n📊 Sample:")
print(sar_features.head(10))

# Save
sar_features.to_csv(FEATURES_DIR / 'sar_features_batch_0_test.csv', index=False)

print(f"\n✓ Saved: features/sar_features_batch_0_test.csv")

print(f"\n🎯 SUCCESS! Ready to scale to 50-date batches!")

FIXED PROCESSING: CRS REPROJECTION
Processing 10 files...
  ✓ 2014-10-15 (None)
  ✓ 2014-12-31 (2014-2016)
  ✓ 2015-04-18 (2014-2016)
  ✓ 2015-07-11 (2014-2016)
  ✓ 2015-12-26 (2014-2016)
  ✓ 2016-05-13 (2014-2016)
  ✓ 2016-07-24 (2014-2016)
  ✓ 2016-09-27 (2014-2016)
  ✓ 2016-12-08 (2014-2016)
  ✓ 2017-02-25 (2014-2016)

✅ Processed!
   Total rows: 1410
   Dates: 10
   Wards: 141

📊 Sample:
         date ward_id period  sar_water_extent  water_pixels  total_pixels
0  2014-10-15    93\n   None               0.0             0         18689
1  2014-10-15    61\n   None               0.0             0          6427
2  2014-10-15    86\n   None               0.0             0          8875
3  2014-10-15    90\n   None               0.0             0         11458
4  2014-10-15    26\n   None               0.0             0          3520
5  2014-10-15    72\n   None               0.0             0          5787
6  2014-10-15   134\n   None               0.0             0          3812
7  20

In [12]:
print("=" * 60)
print("PROCESSING BATCH 1 (50 DATES)")
print("=" * 60)

import rasterio
from rasterio.mask import mask as rasterio_mask

sar_files = sorted(SAR_DIR.glob('sar_water_extent_*.tif'))

print(f"Found {len(sar_files)} files")

all_features = []

for sar_file in sar_files:
    date_str = sar_file.stem.split('_')[-1]
    date_formatted = f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}"
    period = get_period_for_date(date_formatted)

    with rasterio.open(sar_file) as src:
        # Reproject wards to raster CRS
        wards_proj = wards.to_crs(src.crs)

        for idx, ward in wards_proj.iterrows():
            ward_id = ward['WARD']

            try:
                out, _ = rasterio_mask(src, [ward['geometry'].__geo_interface__], crop=True, filled=False)
                data = out[0]
                data = data[~data.mask] if hasattr(data, 'mask') else data
                data = data[np.isfinite(data)]

                if len(data) > 0:
                    water_pct = (data == 1).sum() / len(data)

                    all_features.append({
                        'date': date_formatted,
                        'ward_id': ward_id,
                        'period': period,
                        'sar_water_extent': water_pct
                    })
            except:
                pass

    if len([f for f in all_features if f['date'] == date_formatted]) > 0:
        print(f"  ✓ {date_formatted}")

df = pd.DataFrame(all_features)
df['ward_id'] = df['ward_id'].astype(str).str.strip()

print(f"\n✅ Done! {len(df)} rows")

df.to_csv(FEATURES_DIR / 'sar_features_batch_1.csv', index=False)
print(f"✓ Saved")

BATCH 1: SERVER-SIDE PROCESSING (SMART!)
🎯 Processing 50 dates × 141 wards in Earth Engine
   Export: 1 CSV (not 50 rasters!)
   Expected time: 10-15 min total

Processing 50 dates:
  Range: 2015-02-17 to 2016-11-02
✓ Ward FeatureCollection ready (141 wards)

⚙️  Processing in Earth Engine...


EEException: Collection.loadTable: Collection asset '/Users/romitbasak/Projects/KolkataFloodMapping/data/sar/temp_wards_simple.geojson' not found.

In [13]:
print("=" * 60)
print("UPLOADING WARDS TO EARTH ENGINE (ONE-TIME)")
print("=" * 60)

print("🎯 Strategy: Upload wards as GEE asset")
print("   Then ALL processing can be server-side!")
print("   Future users just reference the asset\n")

# Simplify geometries for faster processing
wards_for_gee = wards.copy()
wards_for_gee['geometry'] = wards_for_gee.geometry.simplify(0.0001)
wards_for_gee = wards_for_gee[['WARD', 'geometry']]
wards_for_gee['WARD'] = wards_for_gee['WARD'].astype(str)

# Save as GeoJSON for upload
gee_wards_file = DATA_DIR / 'wards/kmc_wards_for_gee_asset.geojson'
wards_for_gee.to_file(gee_wards_file, driver='GeoJSON')

print(f"✓ Saved simplified wards: {gee_wards_file}")

print(f"\n📤 UPLOAD TO GEE:")
print(f"""
Option 1: GEE Code Editor (Easy):
  1. Go to: https://code.earthengine.google.com/
  2. Click 'Assets' tab (left panel)
  3. Click 'NEW' → 'Shape files' (or Table upload)
  4. Upload: {gee_wards_file}
  5. Asset name: 'users/YOUR_USERNAME/kmc_wards_141'
  6. Wait 2-5 min for ingestion
  7. Copy asset path

Option 2: Python API (if you prefer):
  # Requires earthengine-api
  # More complex, but can be scripted
""")

print(f"\n⏰ While upload processes:")
print(f"   Let those raster exports finish (use them for Batch 1)")
print(f"   From Batch 2 onwards: Use asset-based server-side approach")

print(f"\n💡 HYBRID TONIGHT:")
print(f"   Batch 1: Use the 50 raster exports (already running)")
print(f"   Batch 2+: Server-side with uploaded asset (much faster!)")

UPLOADING WARDS TO EARTH ENGINE (ONE-TIME)
🎯 Strategy: Upload wards as GEE asset
   Then ALL processing can be server-side!
   Future users just reference the asset

✓ Saved simplified wards: /Users/romitbasak/Projects/KolkataFloodMapping/data/wards/kmc_wards_for_gee_asset.geojson

📤 UPLOAD TO GEE:

Option 1: GEE Code Editor (Easy):
  1. Go to: https://code.earthengine.google.com/
  2. Click 'Assets' tab (left panel)
  3. Click 'NEW' → 'Shape files' (or Table upload)
  4. Upload: /Users/romitbasak/Projects/KolkataFloodMapping/data/wards/kmc_wards_for_gee_asset.geojson
  5. Asset name: 'users/YOUR_USERNAME/kmc_wards_141'
  6. Wait 2-5 min for ingestion
  7. Copy asset path

Option 2: Python API (if you prefer):
  # Requires earthengine-api
  # More complex, but can be scripted


⏰ While upload processes:
   Let those raster exports finish (use them for Batch 1)
   From Batch 2 onwards: Use asset-based server-side approach

💡 HYBRID TONIGHT:
   Batch 1: Use the 50 raster exports (already

In [14]:
print("=" * 60)
print("TESTING EXISTING GEE WARD ASSET")
print("=" * 60)

# Try loading the existing asset
# Common asset names from your files:
asset_options = [
    'users/romitbasak/kmc_wards_141',
    'users/romitbasak/kmc_wards',
    'projects/kolkata-flood-mapping/assets/kmc_wards_141',
    'projects/kolkata-flood-mapping/assets/kmc_wards'
]

for asset_path in asset_options:
    try:
        print(f"\nTrying: {asset_path}...", end='')
        wards_ee = ee.FeatureCollection(asset_path)
        count = wards_ee.size().getInfo()

        print(f" ✅ FOUND!")
        print(f"  Wards: {count}")

        # Quick test - get first ward
        first = wards_ee.first().getInfo()
        print(f"  Properties: {list(first['properties'].keys())}")

        print(f"\n🎯 USE THIS ASSET:")
        print(f"   WARD_ASSET = '{asset_path}'")

        break

    except Exception as e:
        print(f" ❌")

print(f"\nIf none worked, we need to:")
print(f"  1. Delete old asset in GEE Assets tab")
print(f"  2. Re-upload the shapefile")

TESTING EXISTING GEE WARD ASSET

Trying: users/romitbasak/kmc_wards_141... ❌

Trying: users/romitbasak/kmc_wards... ❌

Trying: projects/kolkata-flood-mapping/assets/kmc_wards_141... ❌

Trying: projects/kolkata-flood-mapping/assets/kmc_wards... ✅ FOUND!
  Wards: 141
  Properties: ['WARD']

🎯 USE THIS ASSET:
   WARD_ASSET = 'projects/kolkata-flood-mapping/assets/kmc_wards'

If none worked, we need to:
  1. Delete old asset in GEE Assets tab
  2. Re-upload the shapefile


In [15]:
print("=" * 60)
print("SERVER-SIDE SAR PROCESSING - BATCH 1 (50 DATES)")
print("=" * 60)

# Load ward asset
WARD_ASSET = 'projects/kolkata-flood-mapping/assets/kmc_wards'
wards_ee = ee.FeatureCollection(WARD_ASSET)

print(f"✓ Ward asset loaded: {wards_ee.size().getInfo()} wards")

# Select 50 dates for Batch 1
BATCH_SIZE = 50
batch_1_indices = np.linspace(10, 60, BATCH_SIZE, dtype=int)
batch_1_dates = [all_s1_dates[i] for i in batch_1_indices]

print(f"\n📅 Batch 1: 50 dates")
print(f"   Range: {datetime.fromtimestamp(batch_1_dates[0]/1000).strftime('%Y-%m-%d')} to {datetime.fromtimestamp(batch_1_dates[-1]/1000).strftime('%Y-%m-%d')}")
print(f"   Expected time: 15-20 min (all server-side!)")

# Process all dates
print(f"\n⚙️  Processing 50 dates × 141 wards in Earth Engine...")

all_results = []

for i, date_millis in enumerate(batch_1_dates, 1):
    date_obj = datetime.fromtimestamp(date_millis / 1000)
    date_str = date_obj.strftime('%Y-%m-%d')
    period = get_period_for_date(date_str)

    # Get image and detect water
    s1_img = s1_full.filter(ee.Filter.eq('system:time_start', int(date_millis))).first()
    vv_filt = s1_img.select('VV').focalMedian(100, 'circle', 'meters')
    water = vv_filt.lt(OTSU_S1)

    # Calculate statistics per ward (server-side!)
    ward_stats = water.reduceRegions(
        collection=wards_ee,
        reducer=ee.Reducer.mean(),
        scale=10
    )

    # Get results
    stats = ward_stats.getInfo()

    for feature in stats['features']:
        props = feature['properties']
        all_results.append({
            'date': date_str,
            'ward_id': str(props.get('WARD', '')),
            'period': period,
            'sar_water_extent': props.get('mean', 0)
        })

    if i % 5 == 0:
        print(f"  [{i}/50] {date_str} ({period})...")

# Create DataFrame
batch_1_df = pd.DataFrame(all_results)
batch_1_df['ward_id'] = batch_1_df['ward_id'].astype(str).str.strip()

print(f"\n✅ Batch 1 complete!")
print(f"   Rows: {len(batch_1_df)} (50 dates × 141 wards = 7,050)")
print(f"   Dates: {batch_1_df['date'].nunique()}")
print(f"   Wards: {batch_1_df['ward_id'].nunique()}")

# Save
batch_1_df.to_csv(FEATURES_DIR / 'sar_features_batch_1.csv', index=False)

print(f"\n✓ Saved: features/sar_features_batch_1.csv")

print(f"\n🎯 100% SERVER-SIDE!")
print(f"   No downloads, no disk space, reproducible!")
print(f"\n⏭️  Ready to scale to full 1,290 dates!")

SERVER-SIDE SAR PROCESSING - BATCH 1 (50 DATES)
✓ Ward asset loaded: 141 wards

📅 Batch 1: 50 dates
   Range: 2015-02-17 to 2016-11-02
   Expected time: 15-20 min (all server-side!)

⚙️  Processing 50 dates × 141 wards in Earth Engine...
  [5/50] 2015-04-01 (2014-2016)...
  [10/50] 2015-06-05 (2014-2016)...
  [15/50] 2015-07-23 (2014-2016)...
  [20/50] 2015-11-27 (2014-2016)...
  [25/50] 2016-02-12 (2014-2016)...
  [30/50] 2016-05-13 (2014-2016)...
  [35/50] 2016-06-30 (2014-2016)...
  [40/50] 2016-08-10 (2014-2016)...
  [45/50] 2016-09-15 (2014-2016)...
  [50/50] 2016-11-02 (2014-2016)...

✅ Batch 1 complete!
   Rows: 7050 (50 dates × 141 wards = 7,050)
   Dates: 50
   Wards: 141

✓ Saved: features/sar_features_batch_1.csv

🎯 100% SERVER-SIDE!
   No downloads, no disk space, reproducible!

⏭️  Ready to scale to full 1,290 dates!
